#**Script to obtain the data files that are used to initialize the simulations of transcriptional and translational divergence of yeast duplicated genes**

In [ ]:
import numpy as np
import pandas as pd
from google.colab import files
import io
import scipy.stats as stats

In [ ]:
# Mounting my google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Transcription and translation rates from Hausser et al. (2019) are first imported.

In [ ]:
rates_yeast = pd.read_csv('/content/drive/My Drive/Redaction_SA/Figures_2021_06/Data_init/yeastRates.csv', sep=';', decimal=',')
rates_yeast = rates_yeast.rename(columns={'Unnamed: 0': 'Gene'})

In [ ]:
rates_yeast

,Gene,lm,lp,wRPF,wmRNA,bmEser,amEser,bm,m,bp,cv,apExp,am,isEssential,hasTATA,YEPDFit,pEst
0,TFC3,3.553033,3.064832,1.258158,1.577492,1.125383,0.704432,1.127728,0.423025,2.201503,NaN,-1.057717,0.704302,True,True,0.040325,2.496036
1,VPS8,3.582631,3.105510,0.758155,1.345766,1.146577,0.868402,0.896002,0.191299,1.892547,NaN,NaN,0.704302,False,False,0.020725,1.955355
2,EFB1,2.994317,2.315970,3.849116,3.664817,2.307282,0.058252,3.215053,2.510350,3.453998,NaN,NaN,0.704302,True,False,0.420089,5.835856
3,ERP2,2.811575,2.334454,2.321122,2.327195,1.701383,0.057926,1.877432,1.172729,3.245141,-0.744004,-0.071219,0.704302,False,False,-0.026295,4.289378
4,FUN14,2.775974,2.298853,1.980231,1.698622,0.790021,0.382707,1.248859,0.544156,3.568424,NaN,0.204003,0.704302,False,False,-0.018375,3.984088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4435,OPT2,3.420616,2.943495,-0.236572,0.842609,-0.399593,0.274546,0.392846,-0.311857,1.562993,NaN,NaN,0.704302,False,True,-0.009625,1.122644
4436,SGE1,3.212720,2.735599,1.472025,1.521922,1.158760,0.532465,1.072159,0.367456,2.800172,NaN,NaN,0.704302,False,False,-0.009200,3.039136
4437,ARR1,2.946943,2.469822,1.138934,1.592954,1.449894,0.730236,1.143190,0.438487,2.661827,NaN,NaN,0.704302,False,False,-0.016960,2.971822
4438,ARR2,2.594393,2.117271,0.387390,1.169086,0.400387,0.863932,0.719323,0.014620,2.686701,NaN,NaN,0.704302,False,True,-0.010200,2.572829


## 1) Addition of systematic names

The file saccharomyces_cerevisiae.20200611.gff has been downloaded from Saccharomyces Genome Database (SGD). It contains all genomic features in the S288C reference genome, as of 2020-06-11, as well as the complete genome sequence. It is used to obtain all the systematic - standard name associations.

In [ ]:
features_SGD = pd.read_table('/content/drive/My Drive/Redaction_SA/Figures_2021_06/Data_init/saccharomyces_cerevisiae.20200611.gff', 
                             skiprows=20, header=None, index_col=False, 
                             names=['seqname', 'source', 'feature', 'start', 
                                    'end', 'score', 'strand', 'frame', 
                                    'attribute'], nrows=23067)
# The genome sequence is not read

features_genes = features_SGD[features_SGD['feature'] == 'gene']
features_genes = features_genes.reset_index(drop=True)

In [ ]:
# The features_genes file is used to build a dictionary of standard name -
# systematic name associations

genes_orf = {}

for row in range(features_genes.shape[0]):

  features = features_genes.at[row, 'attribute'].split(';')
  feat_dict = {}

  for feat in features:
    feat_split = feat.split('=')

    feat_dict[feat_split[0]] = feat_split[1]

  if 'gene' in feat_dict.keys(): # For cases when the two IDs are the same 
                                 # (ex: mitochondrial genes)

    genes_orf[feat_dict['gene']] = feat_dict['ID']

  else:

    genes_orf[feat_dict['ID']] = feat_dict['ID']

In [ ]:
# This dictionary is used to add the systematic ID of genes to the rates df
rates_yeast.insert(0, 'ORF', 'problem')
missing = []

for row in range(rates_yeast.shape[0]):
  gene = rates_yeast.at[row, 'Gene']

  if gene in genes_orf.keys():
    rates_yeast.at[row, 'ORF'] = genes_orf[gene]
  
  else:

    missing.append(gene)

print(missing)

['AIM1', 'PHO88', 'DUR1,2', 'ARG5,6', 'FRA2', 'ADE5,7', 'AML1', 'YSP1', 'PET10', 'ENV10', 'MDN1', 'FLD1', 'OSW5', 'ICY2']


In [ ]:
# For the missing genes, corresponding to alternative aliases, the systematic 
# IDs are added manually

missing_ID = ['YAL046C', 'YBR106W', 'YBR208C', 'YER069W', 'YGL220W', 'YGL234W',
              'YGR001C', 'YHR155W', 'YKR046C', 'YLR065C', 'YLR106C', 'YLR404W',
              'YMR148W', 'YPL250C']

count = 0
for gene in missing:

  row = rates_yeast[rates_yeast['Gene'] == gene].index

  rates_yeast.at[row, 'ORF'] = missing_ID[count]

  count += 1

## 2) Addition of duplication-type annotations

Table S1 from Supplementary file 2 of Marchant et al. (2019) is imported. In this file, each yeast ORF (systematic ID) is associated to a duplication type annotation (S: singleton, SSD: small scale duplication,  SSD-successive: successive small-scale duplication, WGD: whole-genome duplication or SSD_WGD: paralogs duplicated by both mechanism of duplication SSD and WGD).

In [ ]:
dupli_type = pd.read_csv('/content/drive/My Drive/Redaction_SA/Figures_2021_06/Data_init/TableS1_Marchant_2019.csv', sep=';')

In [ ]:
rates_yeast['Duplication'] = 'Unspecified' # For cases that are not in
                                           # the dupli_type dataframe

for row in range(rates_yeast.shape[0]):
  orf = rates_yeast.at[row, 'ORF']

  if orf in list(dupli_type['ORF']):
    dupli = dupli_type[dupli_type['ORF'] == orf]['Duplication'].values[0]
    rates_yeast.at[row, 'Duplication'] = dupli

## 3) Recalculation of transcription and translation rates and rates_Hausser.csv export

We recalculated all expression rates reported by Hausser et al. (2019) from the raw data deposited by Weinberg et al. This was done because we mistakenly thought that the calculation of translation rates by Hausser et al. was erroneous. 

These recalculated rates are not used in our work, but some of the preliminary simulations were performed using both the original and recalculated ("corrected") sets of translations rates. In these, the choice of the dataset had no impact on the results.

Transcription and translation rates, as well as protein abundance, are recalculated from the RPKMs reported by Weinberg et al. (2016), following the Methods of Hausser et al. (2019).

Raw data are downloaded from GSE75897 (https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE75897). The RNA-seq RPKM from the RiboZero experiment (GSE75897_RiboZero_RPKMs.txt.gz) and the ribosome profiling RPKMs (GSE75897_RPF_RPKMs.txt.gz) are obtained and imported:

In [ ]:
mRNA_RPKM = pd.read_csv('/content/drive/My Drive/Redaction_SA/Figures_2021_06/Data_init/GSE75897_RiboZero_RPKMs.txt', sep='\t', names=['ORF', 'mRNA RPKM'])
RP_RPKM = pd.read_csv('/content/drive/My Drive/Redaction_SA/Figures_2021_06/Data_init/GSE75897_RPF_RPKMs.txt', sep='\t', names=['ORF', 'RP RPKM'])

In [ ]:
# To obtain a file with only genes for which RPKMs have been reported in
# both experiments

RPKM_all = pd.merge(mRNA_RPKM, RP_RPKM, how='outer')

RPKM_both = RPKM_all.dropna().reset_index(drop=True)

# RPKM of 0.0 are also dropped
RPKM_both = RPKM_both[(RPKM_both['mRNA RPKM'] != 0) & (RPKM_both['RP RPKM'] != 0)].reset_index(drop=True)

There are 5068 genes for which both types of RPKMs are available, compared with 4400 in the dataset from Hausser et al. (2019). This seems consistent with the filtering step described in the Methods of this paper.

In [ ]:
# Constants from Hausser et al. (2019)
alpha_m = 5.1
alpha_p = 1.34
n_mRNA = 60000
n_prot = 1.1e8

# Transcription and translation rates are calculated (Hausser et al., 2019)
RPKM_both['m_est'] = np.log10(n_mRNA*(RPKM_both['mRNA RPKM']/np.sum(RPKM_both['mRNA RPKM'])))
RPKM_both['bm_calc'] = np.log10((10**RPKM_both['m_est']) * alpha_m)

RPKM_both['bp_calc'] = np.log10(((n_prot * alpha_p)/(10**RPKM_both['m_est']))*((RPKM_both['RP RPKM'])/np.sum(RP_RPKM['RP RPKM'])))

# As well as a new estimated protein abundance
RPKM_both['pEst_calc'] = np.log10(((10**RPKM_both['bm_calc']) * (10**RPKM_both['bp_calc'])) / (alpha_m * alpha_p))

In [ ]:
# Merging with the yeastRates dataframe 
rates_corrected = pd.merge(rates_yeast, RPKM_both[['ORF', 'm_est', 'bm_calc', 'bp_calc', 'pEst_calc']], how='left', on=['ORF'])

This is the final rates_Hausser dataframe which was used to initialize the simulations. It is exported:

In [ ]:
rates_corrected.to_csv(path_or_buf='/content/drive/My Drive/Redaction_SA/Figures_2021_06/Data_sim_ready/rates_Hausser.csv', index=False, header=True)

## 4) Reconstitution of paralog couples and calculation of relative divergence

The P1-P2 associations are taken from Table S9 of Supplementary File 2 from Marchant et al. (2019), which compiles both SSD- and WGD-derived paralog pairs (as well as other types of paralogs such as multi-SSD which are ignored in the current work). 


In [ ]:
dupli_pairs = pd.read_csv('/content/drive/My Drive/Redaction_SA/Figures_2021_06/Data_init/TableS9_Marchant_2019.csv', sep=';')
dupli_pairs = dupli_pairs.dropna(subset=['Duplication', 'P1', 'P2']).reset_index(drop=True)

In [ ]:
# A new dataframe is defined
rates_couples = pd.DataFrame(columns=['Duplication', 'ORF_P1', 'ORF_P2'])

# After merging, the dataframe will have all those columns:
# Duplication: Duplication type from Marchant et al.
# ORF_P1: Systematic name paralog 1
# Gene_P1: Standard name paralog 1
# ORF_P2: Systematic name paralog 2
# Gene_P2: Standard name paralog 2
# Couple: P1-P1 (standard names)
# bm_P1: Log10 transcription rate of P1, in mRNAs/h (Hausser et al.)
# bm_P2: Log10 transcription rate of P2, in mRNAs/h (Hausser et al.)
# bp_P1: Log10 Translation rate of P1, in proteins per mRNA per hour (Hausser et al.)
# bp_calc_P1: Recalculated translation rate of P1
# bp_P2: Log10 Translation rate of P2, in proteins per mRNA per hour (Hausser et al.)
# bp_calc_P2: Recalculated translation rate of P2
# pEst_P1: Log10 Estimated protein abundance for P1 (proteins / cell)
# pEst_P2 : Log10 Estiimated protein abundance for P2 (proteins per cell)
# cv_P1: Coefficient of variation for P1 (Newman et al., 2006)
# cv_P2: Coefficient of variation for P2 (Newman et al., 2006)

rates_couples['Duplication'] = dupli_pairs['Duplication']
rates_couples['ORF_P1'] = dupli_pairs['P1']
rates_couples['ORF_P2'] = dupli_pairs['P2']

In [ ]:
# New versions of the rates_corrected dataframe are prepared to be merged with
# the rates_couples df just created

rates_only = rates_corrected[['ORF', 'Gene', 'bm', 'bp', 'cv', 'pEst',
                              'Duplication', 'bp_calc', 'pEst_calc']].copy()

rates_P1 = rates_only.copy()
rates_P1 = rates_P1.rename(columns={'ORF': 'ORF_P1', 'Gene': 'Gene_P1', 
                                    'bm': 'bm_P1', 'bp': 'bp_P1', 'cv': 'cv_P1',
                                    'pEst': 'pEst_P1', 'bp_calc': 'bp_calc_P1',
                                    'pEst_calc': 'pEst_calc_P1'})

rates_P2 = rates_only.copy()
rates_P2 = rates_P2.rename(columns={'ORF': 'ORF_P2', 'Gene': 'Gene_P2', 
                                    'bm': 'bm_P2', 'bp': 'bp_P2', 'cv': 'cv_P2',
                                    'pEst': 'pEst_P2', 'bp_calc': 'bp_calc_P2',
                                    'pEst_calc': 'pEst_calc_P2'})

In [ ]:
# The dataframe is then filled
rates_couples = pd.merge(rates_couples, rates_P1, how='left',
                         on=['Duplication', 'ORF_P1'])

rates_couples = pd.merge(rates_couples, rates_P2, how='left',
                         on=['Duplication', 'ORF_P2'])

In [ ]:
# Keeping only pairs where both paralogs were included in the dataset
rates_couples = rates_couples.dropna(subset=['bm_P1', 'bm_P2', 'bp_P1', 'bp_P2']).reset_index(drop=True)

In [ ]:
# Rearranging the columns in the desired order:
rates_couples = rates_couples[['Duplication', 'ORF_P1', 'Gene_P1', 'ORF_P2', 
                               'Gene_P2', 'bm_P1', 'bm_P2', 'bp_P1', 
                               'bp_calc_P1', 'bp_P2', 'bp_calc_P2', 'pEst_P1', 
                               'pEst_calc_P1', 'pEst_P2', 'pEst_calc_P2', 
                               'cv_P1', 'cv_P2']]

In [ ]:
# Definition of a function to compute log2 fold-changes of relative divergence

def fold_change(prop_P1, prop_P2, data, name):
  """Function to calculate log2 fold-change for a property between two
     duplicates. The two properties are provided as dataframe columns.
     The last argument is used to specify the name of the new column 
     inserted in the data dataframe."""

  df = data.copy()
  df[name] = np.NaN

  for row in range(df.shape[0]):

    value_P1 = df.at[row, f'{prop_P1}']
    value_P2 = df.at[row, f'{prop_P2}']

    if value_P1 >= value_P2:

      df.at[row, name] = (10**value_P1)/(10**value_P2)

    elif value_P1 < value_P2:

      df.at[row, name] = (10**value_P2)/(10**value_P1)

  df[name] = np.log2(df[name])

  return df

In [ ]:
# Calculation of the log2 fold-changes of divergence
rates_logfold = fold_change('bm_P1', 'bm_P2', rates_couples, 'bm_fold_original')
rates_logfold = fold_change('bp_P1', 'bp_P2', rates_logfold, 'bp_fold_original')
rates_logfold = fold_change('pEst_P1', 'pEst_P2', rates_logfold, 'pEst_fold_original')

rates_logfold = fold_change('bp_calc_P1', 'bp_calc_P2', rates_logfold, 'bp_fold_calc')
rates_logfold = fold_change('pEst_calc_P1', 'pEst_calc_P2', rates_logfold, 'pEst_fold_calc')

In [ ]:
# Calculation of normalized differences
rates_div_rel = rates_logfold.copy()

rates_div_rel['bm_diff'] = np.abs(10**rates_div_rel['bm_P2'] - 10**rates_div_rel['bm_P1']) / (10**rates_div_rel['bm_P2'] + 10**rates_div_rel['bm_P1'])
rates_div_rel['bp_diff'] = np.abs(10**rates_div_rel['bp_P2'] - 10**rates_div_rel['bp_P1']) / (10**rates_div_rel['bp_P2'] + 10**rates_div_rel['bp_P1'])
rates_div_rel['pEst_diff'] = np.abs(10**rates_div_rel['pEst_P2'] - 10**rates_div_rel['pEst_P1']) / (10**rates_div_rel['pEst_P2'] + 10**rates_div_rel['pEst_P1'])
rates_div_rel['bp_calc_diff'] = np.abs(10**rates_div_rel['bp_calc_P2'] - 10**rates_div_rel['bp_calc_P1']) / (10**rates_div_rel['bp_calc_P2'] + 10**rates_div_rel['bp_calc_P1'])
rates_div_rel['pEst_calc_diff'] = np.abs(10**rates_div_rel['pEst_calc_P2'] - 10**rates_div_rel['pEst_calc_P1']) / (10**rates_div_rel['pEst_calc_P2'] + 10**rates_div_rel['pEst_calc_P1'])

This is the final dataframe with log2 fold-changes and normalized differences that is used to generate figure 1 as well as to initialize the simulations (and compare the simulated and empirical data at the end). It is exported as csv:

In [ ]:
rates_div_rel.to_csv(path_or_buf='/content/drive/My Drive/Redaction_SA/Figures_2021_06/Data_sim_ready/couples_divergence.csv', index=False, header=True)

The empirical correlations of logfold divergence are calculated:

In [ ]:
r_corr = stats.pearsonr(rates_div_rel['bm_fold_original'], rates_div_rel['bp_fold_original'])
r_calc = stats.pearsonr(rates_div_rel['bm_fold_original'], rates_div_rel['bp_fold_calc'])

rho_corr = stats.spearmanr(rates_div_rel['bm_fold_original'], rates_div_rel['bp_fold_original'])
rho_calc = stats.spearmanr(rates_div_rel['bm_fold_original'], rates_div_rel['bp_fold_calc'])

In [ ]:
r_corr

(0.26115963770858425, 8.38039843945512e-08)

In [ ]:
r_calc

(0.28408216841747713, 4.9587573379420115e-09)

In [ ]:
rho_corr

SpearmanrResult(correlation=0.2781015739828044, pvalue=1.0635357565012332e-08)

In [ ]:
rho_calc

SpearmanrResult(correlation=0.2916321433457395, pvalue=1.8437959466822292e-09)

Correlations with are also calculated from log2 fold-changes computed in randomized orientations (only for original data)

In [ ]:
rates_rand = rates_div_rel[['bm_P1', 'bm_P2', 'bp_P1', 'bp_P2']].copy()
rates_rand = 10**rates_rand
rng = np.random.default_rng()
corr_model = pd.DataFrame(columns=['pearson', 'r_pval', 'spearman', 'rho_pval'])
corr_rand = corr_model.copy()

In [ ]:
for i in range(50):
  log_rand = rates_rand.copy()
  
  assos1 = rng.integers(0, 2, log_rand.shape[0])
  assos2 = np.abs(assos1 - 1)
  
  P1_rand = log_rand.copy()
  P2_rand = log_rand.copy()
  
  P1_rand['Logfold_Bm'] = P1_rand['bm_P1'] / P1_rand['bm_P2']
  P1_rand['Logfold_Bp'] = P1_rand['bp_P1'] / P1_rand['bp_P2']
  P2_rand['Logfold_Bm'] = P2_rand['bm_P2'] / P2_rand['bm_P1']
  P2_rand['Logfold_Bp'] = P2_rand['bp_P2'] / P2_rand['bp_P1']
  
  P1_rand = P1_rand * assos1[:, np.newaxis]
  P2_rand = P2_rand * assos2[:, np.newaxis]

  log_rand = P1_rand.copy()
  
  log_rand = log_rand + P2_rand
  
  log_rand['Logfold_Bm'] = np.log2(log_rand['Logfold_Bm'])
  log_rand['Logfold_Bp'] = np.log2(log_rand['Logfold_Bp'])

  corr_data = corr_model.copy()
  pearson = stats.pearsonr(log_rand['Logfold_Bm'], log_rand['Logfold_Bp'])
  spearman = stats.spearmanr(log_rand['Logfold_Bm'], log_rand['Logfold_Bp'])

  corr_data.at[0, 'pearson'] = pearson[0]
  corr_data.at[0, 'r_pval'] = pearson[1]
  corr_data.at[0, 'spearman'] = spearman[0]
  corr_data.at[0, 'rho_pval'] = spearman[1]

  corr_rand = pd.concat([corr_rand, corr_data])

corr_rand = corr_rand.reset_index(drop=True)

In [ ]:
# Median pearson correlation coefficient:
corr_rand['pearson'].median()

0.43356900636988405

In [ ]:
# Median pearson correlation p-values:
corr_rand['r_pval'].median()

3.564185391413534e-20

In [ ]:
# Median spearman correlation coefficient:
corr_rand['spearman'].median()

0.43367207697677435

In [ ]:
# Median spearman correlation p-values:
corr_rand['rho_pval'].median()

3.484423291682522e-20

Last thing: How often is the most expressed copy also the most transcribed / translated?

In [ ]:
P1_most = np.where(rates_div_rel['pEst_P1'] > rates_div_rel['pEst_P2'], 1, 0)
P2_most = np.where(rates_div_rel['pEst_P2'] > rates_div_rel['pEst_P1'], 1, 0)

bm_P1 = np.where(rates_div_rel['bm_P1'] >= rates_div_rel['bm_P2'], 1, 0) * P1_most
bm_P2 = np.where(rates_div_rel['bm_P2'] > rates_div_rel['bm_P1'], 1, 0) * P2_most
bm_freq = (np.sum(bm_P1) + np.sum(bm_P2)) / rates_div_rel.shape[0]

bp_P1 = np.where(rates_div_rel['bp_P1'] >= rates_div_rel['bp_P2'], 1, 0) * P1_most
bp_P2 = np.where(rates_div_rel['bp_P2'] > rates_div_rel['bp_P1'], 1, 0) * P2_most
bp_freq = (np.sum(bp_P1) + np.sum(bp_P2)) / rates_div_rel.shape[0]

In [ ]:
print(bm_freq, bp_freq)

0.863080684596577 0.7677261613691931


In [ ]:
rates_div_rel

,Duplication,ORF_P1,Gene_P1,ORF_P2,Gene_P2,bm_P1,bm_P2,bp_P1,bp_calc_P1,bp_P2,bp_calc_P2,pEst_P1,pEst_calc_P1,pEst_P2,pEst_calc_P2,cv_P1,cv_P2,bm_fold_original,bp_fold_original,pEst_fold_original,bp_fold_calc,pEst_fold_calc,bm_diff,bp_diff,pEst_diff,bp_calc_diff,pEst_calc_diff
0,SSD,YAL036C,RBG1,YGR173W,RBG2,1.718675,1.643553,3.092909,3.350873,3.097578,3.354366,3.978390,4.237767,3.907936,4.166139,-0.852169,-0.871762,0.249550,0.015510,0.234040,0.011605,0.237945,0.086272,0.005375,0.080935,0.004022,0.082279
1,SSD,YAL039C,CYC3,YKL087C,CYT2,0.891671,1.082354,3.449659,3.570784,3.456405,3.498349,3.508135,3.630675,3.705564,3.748922,NaN,NaN,0.633434,0.022409,0.655843,0.240625,0.392809,0.216071,0.007766,0.223463,0.083201,0.135302
2,SSD,YAL042W,ERV46,YML067C,ERV41,1.970836,1.617977,2.782261,3.091116,2.991756,3.229293,3.919902,4.230172,3.776538,4.015489,NaN,-0.666150,1.172172,0.695928,0.476245,0.459012,0.713160,0.385279,0.236620,0.163571,0.157753,0.242249
3,SSD,YAL060W,BDH1,YAL061W,BDH2,1.823724,0.887096,2.972863,3.245824,3.073859,3.384797,3.963392,4.237767,3.127760,3.440113,-0.580871,NaN,3.111409,0.335500,2.775910,0.461658,2.649751,0.792580,0.115754,0.745193,0.158647,0.725108
4,SSD,YBL002W,HTB2,YDR224C,HTB1,2.552705,2.507437,3.724892,3.535228,3.833681,3.644017,5.444402,5.256152,5.507923,5.319673,-0.951170,NaN,0.150378,0.361389,0.211011,0.361389,0.211011,0.052070,0.124597,0.073001,0.124597,0.073001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,WGD,YIL135C,VHS2,YNL074C,MLF3,1.082863,1.264818,2.931731,3.261975,2.957714,3.303574,3.181400,3.513058,3.389337,3.736611,-0.666351,NaN,0.604439,0.086313,0.690751,0.138190,0.742629,0.206471,0.029905,0.234925,0.047857,0.251839
405,WGD,YMR177W,MMT1,YPL224C,MMT2,0.959662,1.090190,2.711851,3.110034,2.656638,3.032142,2.838319,3.237916,2.913634,3.290551,NaN,NaN,0.433605,0.183414,0.250190,0.258753,0.174852,0.149155,0.063481,0.086493,0.089437,0.060525
406,WGD,YOR237W,HES1,YPL145C,KES1,0.238656,1.932001,2.694922,3.023173,2.939815,3.268066,2.100384,2.430049,4.038621,4.368286,NaN,-0.920276,5.625170,0.813515,6.438685,0.813515,6.438685,0.960283,0.274702,0.977206,0.274702,0.977206
407,WGD,YGR180C,RNR4,YJL026W,RNR2,2.387377,2.181690,3.213313,3.442151,3.102431,3.394253,4.767495,4.997747,4.450927,4.744163,-0.908685,-0.942334,0.683274,0.368342,1.051616,0.159114,0.842388,0.232476,0.126969,0.349139,0.055089,0.283928
